In [1]:
#School of Mechanical Engineering - State University of Campinas
#Paulo Yoshio Kuga
#First Release: October 2023

#Present version: 20231001

#this is the main program to run neural network identification 

using LinearAlgebra
using ControlSystems
using Plots
using ControlSystemIdentification
using Random
using Statistics
using DelimitedFiles
using CUDA
using Flux
using Hyperopt


include("KugaPack.jl")
include("sysid.jl")
include("blockpack.jl")
include("analysis.jl")

n4sidPatternArg (generic function with 1 method)

In [2]:
#numerical conditions are set up
dt = 2.5e-4;
maxT = 3;
ns = round(Int,maxT/dt)+1;
t = 0:dt:maxT;

#problem conditions are set up
x0 = [0; 0; 0; 0;];

w = 6
U = chirp(w,t)'#ones(1,ns)#chirp(w,t)';#0.01*PulseGen(ns); 

#using the given parameters, the system (discrete space state) is created 
Param = [0.77 0.59 2.1 1.2 9 200 200 200];
sysP,sysD = F2DOF(dt,Param);

Y,_,_= lsim(sysD,U,t,x0=x0); #time response
magD,phaseD,w = bode(sysD); #frequency response


In [3]:
#Since F2DOF is a two degree freedom problem, we can state that the number of outputs is 2
#This implies the noise beign as (2,n)
#In this analysis, we are using MersenneTwister as the random number generator. 

seed = 98832+5556594 #seed for mersenne twister
noise = noise_gen(seed,2,ns);

fineza = 1;

NAmp = [i*1e-2 for i in 0:fineza:100];

#identification parameters
nx = 4;
p = 700; #round(Int,ns/2);
na = 4;
nb = 1;
ny = 2;
nu = 1;
it = 200;

In [4]:
x0 = [0; 0]
H, Yt = DataNARX(Y,U, ny, nu, na, nb,ns);
Data = zip(H,Yt);

p = 500
model = NNARX_LSTM(ny,nu,na,nb,p)

opt_state = Flux.setup(Adam(), model.nn); 
Flux.train!(model.nn, Data, opt_state) do m, x, y
        Flux.mse(m(x), y);
end

In [7]:
Ypred = nnlsim(model, U, x0);

plot(t,[Y' Ypred'])


ErrorException: Scalar indexing is disallowed.
Invocation of getindex resulted in scalar indexing of a GPU array.
This is typically caused by calling an iterating implementation of a method.
Such implementations *do not* execute on the GPU, but very slowly on the CPU,
and therefore are only permitted from the REPL for prototyping purposes.
If you did intend to index this array, annotate the caller with @allowscalar.